In [1]:
import numpy as np
import pandas as pd
import pdb
#import time

In [2]:
meteo_vars = ['pw','t850','v850','u850','v300','u300','z1000','z500','z300']
#Feature set at time st
st = [pd.DataFrame([]), pd.DataFrame([]), pd.DataFrame([]), pd.DataFrame([])]
# Total number of locations
locations = 5328
# Historical data time frame
time = 11300
# Offset of days
offset = 6
#Data frame of feature attributes, c
c = pd.DataFrame(np.load('./sample_data/provided/target_1980_2010.npy')[:,0], columns=['target'])

In [17]:
# note: zs function has been tested
# Takes two pandas frames as arguments
def zs(crf1, crf2):
	#pdb.set_trace()
	# Pearson correlation coefficient, r
	r = crf1[crf1.columns[0]].corr(crf2[crf2.columns[0]],method='kendall')
	#print('Pearson coefficient %f' % (r))
	# Fisher Z transformation on r'
	rp = np.arctanh(r)
	#print('Fisher Z trans %f' % (rp))
	# Standard Error Calculation for transformed correlation r'
	N = time
	srp = 1/((N - 3.0)**(0.5))
	#print('Z-score: %f' %(rp/srp))
	return rp/srp

In [4]:
def disc_prune(fi, m_b):

	#iterate across the features stored in the selected feature set
	
	# need to iterate from top of array since we're deleting elements as we go
	col_ind = m_b.columns.values
	
	for column in m_b:
		y = pd.DataFrame(m_b[column])
		
		# This checks for redundancy of the feature
		if abs(zs(y, c)) > abs(zs(fi, c)) and abs(zs(fi, y)) >= abs(zs(fi, c)):
			return False
		
		# Newly found fi makes Y redundant -- fi is closer to the target feature
		# than fi
		if abs(zs(fi, c)) > abs(zs(y, c)) and abs(zs(fi, y)) >= abs(zs(fi, c)):
			m_b.drop(column, axis=1, inplace=True)

In [18]:
markov_b = pd.DataFrame([])
#tic = time.time()
for var in range(0, 9):
	data = np.load('./sample_data/generated/npy_format/preproc_data2/%s.npy' % (meteo_vars[var]))
	
	print('Processing feature type %s' % (meteo_vars[var]))
	
	for day in range(0, 4):
		
		print('Processing day %d' % (day))
		
		ddata = data[offset+day:time+offset+day,:]
		
		for loc in range(0, locations):
			colname = '%d_%s_%d' % (loc, meteo_vars[var], day)
			
			#print('processing %s: ' % (colname))
			
			f_i = pd.DataFrame(ddata[:,loc], columns=[colname])
			
			if abs(zs(c, f_i)) < 1.96: 
				continue
				
			if (disc_prune(f_i, markov_b) == False):
				continue
			
			markov_b = pd.concat([markov_b, f_i], axis=1)
		
		print('Finished processing day %d' % (day))
		
		
	#pdb.set_trace()
	print('Finished processing feature type %s' % (meteo_vars[var]))
		
			


#toc = time.time()
#print "operation took %f minutes" % ((toc - tic)/60)

Processing feature type pw
Processing day 0
Finished processing day 0
Processing day 1
Finished processing day 1
Processing day 2
Finished processing day 2
Processing day 3
Finished processing day 3
Finished processing feature type pw
Processing feature type t850
Processing day 0
Finished processing day 0
Processing day 1
Finished processing day 1
Processing day 2
Finished processing day 2
Processing day 3
Finished processing day 3
Finished processing feature type t850
Processing feature type v850
Processing day 0
Finished processing day 0
Processing day 1
Finished processing day 1
Processing day 2
Finished processing day 2
Processing day 3
Finished processing day 3
Finished processing feature type v850
Processing feature type u850
Processing day 0
Finished processing day 0
Processing day 1
Finished processing day 1
Processing day 2
Finished processing day 2
Processing day 3
Finished processing day 3
Finished processing feature type u850
Processing feature type v300
Processing day 0
Fi

In [19]:
markov_b.shape

(11300, 19)

In [7]:
markov_b.head()

,1209_t850_1,1356_v850_0,3661_v850_2,3906_v850_3,3723_u850_2,1729_u850_3,3451_u850_3,2030_v300_0,3587_v300_0,4365_v300_0,1257_v300_1,1168_u300_2,4035_u300_2,606_z1000_3,4944_z1000_3
0,282.46,-3.290000,0.630010,17.96,0.000008,1.08000,5.96,2.53,-9.07000,1.230000,-1.25000,25.21,-2.54,288,175
1,282.30,0.010008,0.430010,-6.42,6.710000,0.80001,6.80,3.63,-7.22000,-2.920000,-2.75000,24.18,-19.25,197,179
2,282.16,1.310000,-0.369990,7.91,5.180000,4.80000,1.88,5.33,0.16001,-4.470000,-2.49000,31.48,-20.82,234,167
3,282.58,3.230000,-0.769990,7.68,1.980000,3.93000,-2.09,5.33,0.13001,-4.040000,0.63001,16.66,-15.39,214,174
4,282.85,0.610010,0.060008,2.88,2.880000,2.33000,6.33,1.43,0.81001,0.010008,0.35001,16.38,-17.34,137,164


In [8]:
#let's save blanket for pearson correlation 
# there are 15 features
markov_b.to_csv(r'./sample_data/generated/relevant_features/pearson.csv')

In [16]:
#let's save blanket for spearman correlation 
markov_b.to_csv(r'./sample_data/generated/relevant_features/spearman.csv')